#   Данные об оттоке клиентов некоторого банка

Для разработки кампании лояльностипо удержанию клиентов необходимо выяснить основные причины оттока клиентов. Иными словами, нужно установить, чем ушедшие клиенты отличаются от лояльных и как между собой связаны различные признаки, определяющие клиентов.

In [165]:
import pandas as pd
churn_data = pd.read_csv('data/churn.csv')
churn_data.drop('RowNumber', axis=1, inplace=True)
churn_data['string_exited'] = churn_data['Exited'].apply(lambda x : 'Ушедший' if x==1 else 'Лояльный') 
churn_data['string_IsActiveMember'] = churn_data['IsActiveMember'].apply(lambda x : 'Активный' if x==1 else 'Не активный') 
# Этот признак добавляется только для построения графиков. Для моделей удобнее использовать числовые признаки.
display(churn_data)

CustomerId    Surname  CreditScore Geography  Gender  Age  Tenure  \
0       15634602   Hargrave          619    France  Female   42       2   
1       15647311       Hill          608     Spain  Female   41       1   
2       15619304       Onio          502    France  Female   42       8   
3       15701354       Boni          699    France  Female   39       1   
4       15737888   Mitchell          850     Spain  Female   43       2   
...          ...        ...          ...       ...     ...  ...     ...   
9995    15606229   Obijiaku          771    France    Male   39       5   
9996    15569892  Johnstone          516    France    Male   35      10   
9997    15584532        Liu          709    France  Female   36       7   
9998    15682355  Sabbatini          772   Germany    Male   42       3   
9999    15628319     Walker          792    France  Female   28       4   

        Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0          0.00              1          1               1        101348.88   
1      83807.86              1          0               1        112542.58   
2     159660.80              3          1               0        113931.57   
3          0.00              2          0               0         93826.63   
4     125510.82              1          1               1         79084.10   
...         ...            ...        ...             ...              ...   
9995       0.00              2          1               0         96270.64   
9996   57369.61              1          1               1        101699.77   
9997       0.00              1          0               1         42085.58   
9998   75075.31              2          1               0         92888.52   
9999  130142.79              1          1               0         38190.78   

      Exited string_exited string_IsActiveMember  
0          1       Ушедший              Активный  
1          0      Лояльный              Активный  
2          1       Ушедший           Не активный  
3          0      Лояльный           Не активный  
4          0      Лояльный              Активный  
...      ...           ...                   ...  
9995       0      Лояльный           Не активный  
9996       0      Лояльный              Активный  
9997       1       Ушедший              Активный  
9998       1       Ушедший           Не активный  
9999       0      Лояльный           Не активный  

[10000 rows x 15 columns]

### Признаки:
- CustomerId — идентификатор клиента
- Surname — фамилия клиента
- CreditScore — кредитный рейтинг клиента (чем он выше, тем больше клиент брал кредитов и возвращал их)
- Geography — страна клиента (банк международный)
- Gender — пол клиента
- Age — возраст клиента
- Tenure — сколько лет клиент пользуется услугами банка
- Balance — баланс на счетах клиента в банке
- NumOfProducts — количество услуг банка, которые приобрёл клиент
- HasCrCard — есть ли у клиента кредитная карта (1 — да, 0 — нет)
- IsActiveMember — есть ли у клиента статус активного клиента банка (1 — да, 0 — нет)
- EstimatedSalary — предполагаемая заработная плата клиента
- Exited — статус лояльности (1 — ушедший клиент, 0 — лояльный клиент)

In [3]:
import plotly
import plotly.express as px
plotly.__version__

'5.1.0'

## Задание 9.1
Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

In [4]:
# exited_data = churn_data['Exited'].value_counts().sort_index()
# 
# temp = pd.DataFrame({
#     'Статус лояльности': ['лояльный клиент', 'ушедший клиент'],
#     'Количество клиентов': exited_data.values
# })

fig = px.pie(churn_data, names='string_exited', title='Соотношение ушедших и лояльных клиентов')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### Вывод: 
Соотношение лояльных клиентов к ушедшим примерно 80 к 20. Судить о том, много это или мало, нужно в сравнении с другими банками.
Предварительно можно сделать вывод, что большая часть клиентов всё же осталась лояльной, что неплохо. 
Далее стратегию можно разделить на 2 направления: удержание лояльных клиентов и возврат ушедших.
<hr>

## Задание 9.2
Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

In [5]:
balance_data = churn_data[churn_data['Balance'] > 2500]
fig = px.histogram(balance_data, x='Balance')
fig.show()

### Вывод
Смотря на гистограмму распределения баланса клиентов, можно сделать несколько выводов:
* данные имеют нормальное распределение, ярко выраженные аномалии отсутствуют. Это говорит нам о качестве представленных данных. Статистический анализ таких данных упрощается, результатам анализа будет больше доверия.
* у гистограммы имеется ярко выраженный пик, другими словами она унимодальна. Это обычное явление для распределения даных одной популяции.
* пик графика приходится на 122-124к, в более широком обхвате: 96-140к. Эти данные указывают на наиболее часто встречающуюся платёжеспособность клиентов, что можно использовать при построении, к примеру. маркетинговой стратегии компании.
* минимальное значение = 22к, максимальное 224к. Также есть несколько выбросов в ту и другую сторону, что нужно учитывать при построении модели. Дальнейший анализ должен показать, можно ли использовать эти данные или лучше очистить.
<hr>

## Задание 9.3
Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [6]:
fig = px.histogram(churn_data, x='Balance', #log_y=True,
                   color='string_exited')
fig.show()

### Вывод
По графику можем сделать вывод, что в среднем у лояльных клиентов на балансе больше средств, чем у ушедших. При этом распределение нормальное у обоих групп, что свидетельствует о качестве наблюдаемых данных.
Также видна ярко выраженная аномалия для нулевого баланса клиентов. Явно видно, что у многих клиентов есть счёт в банке, но они им не пользуются.
<hr>

## Задание 9.4
Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [18]:
fig = px.box(churn_data, x='Age', y='string_exited')
fig.show()

### Вывод
Лояльные клиенты в среднем моложе ушедших: по квартилям мы видим возраст в 31-41 у лояльных клиентов против 38-51 у ушедших клиентов. Банку явно следует обратить внимание на эту возрастную категорию. 
При этом в данных по лояльным клиентам больше потенциальных выбросов, что может свидетельствовать как о большем разбросе клиентской аудитории банка, так и о возможном недобросовестном или заведомо ложном заполнении персональных данных от клиентов. Скорее всего потребуется дополнительная очистка данных.
<hr>

## Задание 9.5
Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [11]:
fig = px.scatter(churn_data, x='CreditScore', y='EstimatedSalary', color='string_exited')
fig.show()

### Вывод
Судя по диаграмме рассеивания, кредитный рейтинг никак не связан с предполагаемой зарплатой клиента, даже в разрезе данных по оттоку клиентов. Значения распределены по всему графику без каких-либо взаимосвязей. 
Единственный вывод можно сделать по данным о низком кредитном рейтинге. Кредитный рейтинг ниже 400 баллов наблюдается только у ушедших клиентов, но по сравнению с остальными их довольно мало.
<hr>

## Задание 9.6 
Кто чаще уходит, мужчины или женщины? Постройте график, который иллюстрирует это.

Процент ушедших клиентов в каждой группе можно рассчитать как среднее по столбцу Exited (так как 1 — это ушедшие клиенты, а 0 — лояльные, среднее арифметическое по столбцу обозначает долю ушедших клиентов).

In [196]:
fig = px.sunburst(
    churn_data
    , path=['Gender','string_exited']
    , width=700
    , title='Кто чаще уходит, мужчины или женщины?'
)
fig.update_traces(textinfo='label+percent parent') 

fig.show()

### Вывод
По представленным данным чаще уходят в другие банки женщины, чем мужчины: 25% против 16%. 
Это следует учитывать в маркетинговой кампании. 
<hr>

## Задание 9.7
Как отток клиентов зависит от числа приобретённых у банка услуг? Для ответа на этот вопрос постройте многоуровневую столбчатую диаграмму.

In [151]:
grouped_data = churn_data.groupby(by=['NumOfProducts','Exited'], as_index=False)[['CustomerId']].count()
#display(grouped_data)
fig = px.bar(grouped_data
             , x='NumOfProducts' 
             , y='CustomerId'
             , width=700
             , color='Exited'
             , barmode='group'
             )
fig.show()

### Вывод
....
<hr>

## Задание 9.8
Как влияет наличие статуса активного клиента на отток клиентов? Постройте диаграмму, иллюстрирующую это. 

Что бы вы предложили банку, чтобы уменьшить отток клиентов среди неактивных?

In [199]:
fig = px.sunburst(
    churn_data
    , path=['string_IsActiveMember', 'string_exited']
    , width=700
    , title='Влияет ли статус активного клиента на отток клиентов?'
)
fig.update_traces(textinfo='label+percent entry') 

fig.show()

### Вывод
....
<hr>

## Задание 9.9
В какой стране доля ушедших клиентов больше? Постройте тепловую картограмму, которая покажет это соотношение на карте мира. Предположите, с чем это может быть связано.

### Вывод
...
<hr>

## Задание 9.10
Переведите числовой признак CreditScore в категориальный. Для этого воспользуйтесь функцией get_credit_score_cat(), которая приведена ниже. Примените её к столбцу CreditScore и создайте новый признак CreditScoreCat — категории кредитного рейтинга.

Постройте сводную таблицу, строками которой являются категории кредитного рейтинга (CreditScoreCat), а столбцами — количество лет, в течение которых клиент пользуется услугами банка (Tenure). В ячейках сводной таблицы должно находиться среднее по признаку оттока (Exited) — доля ушедших пользователей.

На основе полученной сводной таблицы постройте тепловую карту с аннотацией. Найдите на тепловой карте категории клиентов, которые уходят чаще всего.